# Intro

This notebook introduces some of the basics of Julia, including necessary syntax, multiple dispatch, some common applications, plotting, and benchmarking.
It is not a comprehensive introduction to the language, but hopes to be a decent reference for "hope do I do [that thing] again?"
In this notebook we'll use a few libraries, but the needed install commands are included:
`PyPlot.jl`, `DataFrame.jl`, ..

# Syntax Basics

Here we'll cover some of the basic Julia syntax for doing things like conditionals, loops, and functions.

In [ ]:
# Comments like python
# If statements

# Notice: dynamic typing
x = 1
y = 2

if ( x > y )
    println("x > y") # println is print with an added new line
elseif x == y  # These () are not needed 
println( "x = y" ) # Indentation is not necessary
else
    println("x < y")
end

In [ ]:
# There is also a short form, ternary operator
# if x > y then x else y
(x > y) ? x : y

In [ ]:
# For loops!
# Note that unlike, Python, in Julia for loops are FAST. There is no reason to avoid them, at all.

# sum even numbers between 1 and 100 (inclusive)
s = 0.0
for i in 1:100 # this is inclusive, unlike Python
    if (i % 2 == 0)
        s += i
    end
end
s

In [ ]:
# While loop, same example

s = 0.0
i = 1
while ( i <= 100 ) # As expected
    if (i % 2 == 0)
        s += i
    end
    i += 1
end
s

In [ ]:
# Let's turn the previous stuff into a function!

"""
Julia docstring.
This functions sums evens in [start, stop]
"""
function SumEvens( start, stop )
    s = 0.0
    for i in start:stop
        if (i % 2 == 0)
            s += i
        end
    end
    return s 
    # Note: you don't need a return statement. 
    # The default "return" is the last accessed variable. Having a return is good form, though.
end

In [ ]:
SumEvens( 1, 100 ) # nice

In [ ]:
?SumEvens

In [ ]:
# A note on strings: you must use double quotes ""
# Single quotes are for characters, only.

my_string = "This is a string"
my_char = 'c'

Booleans use lowercase

In [ ]:
flag = true

Julia is less forgiving with array operations than Python

In [ ]:
array1 = [ 1.0, 2.0, 3.0 ]
array2 = [ 30.0, 40.0, 50.0 ]

# array1 * array2
array1 .* array2 # .* is an example of elementwise operations. Can do this with most operators, and with functions.
SumEvens.(array1, array2) # the output is an array (SumEvens(array1[1], array2[1]), ...)

# A note: It is possible that broadcasting operations like this may be slower than a for loop.

Arrays in Julia can be resized using `push!(array, newval)`. The `!` at the end of the function name is Julia notation indicating that this function modifys the input argument (here, by appending `newval` to the end).

In [ ]:
array = [] # empty array
for i in 1:10
    push!(array, i)
end

println(array)

In [ ]:
# similarly, pop! removes the last item
for i in 1:5
    pop!(array)
end

In [ ]:
println(array)

# Static Typing and Multiple Dispatch

In all of the previous examples we made use of dynamic typing -- we never told Julia what types variables would take, even for our functions.
For functions, Julia has the ability to implement static typing, allowing us (and the compiler) to get some extra mileage. 
Let's dig into this.

Let's consider a function that creates an array of input size `n` of random values between `a` and `b`.

In [ ]:
"""
Dynamic typing.
Notice we can set default argument values just like in Python
"""
function SumRands( n, a=0.0, b=1.0 )
    array = rand( n ) # array of n rands. 
    
    # rand gives between 0 and 1. map to between a and b,
    # @inbounds is a macro to disable array bounds checking. Faster
    @inbounds for i in 1:n
       array[i] = array[i] %((b - a) + 1.0) + a
    end
    # Here we could do array .% ((b - a) + 1.0) + a, but the .% is slower
    
    s = 0.0
    @inbounds for i in 1:n
        s += array[i]
        # @inbounds is a macro to disable bounds checking. Can be faster
    end
    
    return s
end

In [ ]:
SumRands( 10000 ) # Notice we only passed the first arg, the others defaulted.

The above used dynamic typing, but we can be more careful

In [ ]:
"""
Static typing
"""
function SumRands_Static( n::Int64, a::Int64 = 0, b::Int64 = 1 ) # var::Type
    array :: Array{Float64,1} = rand( n ) # array of n rands. Array{Type, Dims}
    
    # rand gives between 0 and 1. map to between a and b,
    for i in 1:n
       array[i] = array[i] %((b - a) + 1.0) + a
    end
    # Here we could do array .% ((b - a) + 1.0) + a, but the .% is slower
    
    s :: Float64 = 0.0
    # @inbounds is a macro to disable bounds checking. Can be faster
    @inbounds for i in 1:n
        s += array[i]
    end
    
    return s
end

"""
Static typing
"""
function SumRands_Static( n::Int64, a::Float64 = 0.0, b::Float64 = 1.0 ) # var::Type
    array :: Array{Float64,1} = rand( n ) # array of n rands. Array{Type, Dims}
    
    # rand gives between 0 and 1. map to between a and b,
    for i in 1:n
       array[i] = array[i] %((b - a) + 1.0) + a
    end
    # Here we could do array .% ((b - a) + 1.0) + a, but the .% is slower
    
    s :: Float64 = 0.0
    @inbounds for i in 1:n
        s += array[i]
        # @inbounds is a macro to disable bounds checking. Can be faster
    end
    
    return s
end

In [ ]:
SumRands_Static(100, 0, 1)

In [ ]:
SumRands_Static(100, 0.0, 1.0)

In [ ]:
SumRands_Static(100, 0, 1.0)

In [ ]:
methods(SumRands_Static)

This example was a bit contrived, but you can imagine scenarios where we will want various methods. For example, a `ComputePressure` equation of state call that can take scalar arguments or vector arguments, or different sets of args if its supposed to call a gamma law EoS vs a tabulated one.

# Some Linear Algebra
See [Julia's linear algebra tutorial](https://web.mit.edu/julia_v0.6.2/julia/share/doc/julia/html/en/stdlib/linalg.html) and [documentation/tutorial](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/).

Most of these functions make use of LAPACK

In [ ]:
using LinearAlgebra
include("funcs.jl") # Include a user made file for Hilbert matrices

In [ ]:
A = Hilbert_static( 5 )
B = ones( 5, 5 )
B += Diagonal( B )
b = A * ones(5)
C = [1.5 2 -4; 3 -1 -6; -10 2.3 4] # We can define like this too

In [ ]:
# Some basic operations
det(B)

In [ ]:
tr(B)

In [ ]:
inv(B)

In [ ]:
# Julia has a built in identity operator I. No need to define our own!
B * I - B

In [ ]:
w,v = eigen(B) # can also use eigvals(B), eigvecs(B)

In [ ]:
# Ex: Test how the condition number degrades as the size of the Hilbert matrix increases


Let's solve Ax = b. First up is to factorize A. we can use `factorize()` to do this -- it selects the appropriate choice (LU, Cholesky, etc). [Documentation](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.factorize)

In [ ]:
factorize(A) # It does Cholesky! Note that it returns a "factorization object"

# Aside: If you are trying to be memory efficient, try cholesky!(A). Functions that end in "!"
# overwrite the inputs - many common functions have these forms available.

In [ ]:
U = factorize(A);

To solve linear systems in Julia, we use the `\` operator (see also [ldiv](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.ldiv!)). This is a "polyalgorithm" that, like factorize, will choose the ideal solver based on the inputs, such as forward substitution.

In [ ]:
x = U\b

# Real Data, Plotting, etc

Let's load and plot some real data. We'll use a progenitor model from [Sukhbold et al. (2016)](https://ui.adsabs.harvard.edu/abs/2016ApJ...821...38S/abstract). The data are included here and available [online](http://doi.org/10.17617/1.b).

We use [DelimitedFiles: readdlm](https://docs.julialang.org/en/v1/stdlib/DelimitedFiles/) here for reading the input. It works well, but doesn't unpack for you as far as I can tell.
The external package [CSV.jl](https://csv.juliadata.org/stable/) has more functionality, including reading into DataFrames, and may be worth looking into.

In [ ]:
using DelimitedFiles
fn = "./s9.0_presn.FLASH" 

In [ ]:
data = readdlm(fn, skipstart=30, header=false)
# radius mass dens temp pres velx velz ye neut H1 He3 He4 C12 N14
# O16 Ne20 Mg24 Si28 S32 Ar36 Ca40 Ti44 Cr48 Cr56 Fe52 Fe54 Ni56 Fe56

If you want, we can load this into a nicer data structure using [DataFrames.jl](https://dataframes.juliadata.org/stable/)

In [ ]:
# using Pkg; Pkg.add("DataFrames")
using DataFrames
keys = [ "radius", "mass", "dens", "temp", "pres", "velx", "velz", "ye", "neut", "H1", "He3", "He4", "C12", "N14",
"O16", "Ne20", "Mg24", "Si28", "S32", "Ar36", "Ca40", "Ti44", "Cr48", "Cr56", "Fe52", "Fe54", "Ni56", "Fe56" ];

df = DataFrame()
for i in 1:length(data[1,:])
    df[!,keys[i]] = data[:,i] # Syntax a bit odd.. may be a nicer way
end

In [ ]:
df.radius; # or df[!,"radius"]
names(df); # to access cols

We can also make a user defined type to hold this data. There's lots that can be done with structs to emulate some of the features of object oriented programming.

In [ ]:
struct Progenitor
    radius  :: Vector{Float64}
    density :: Vector{Float64}
end
s9 = Progenitor(df.radius, df.dens);
s9.density;

Let's plot! We'll start with Julia's [default](https://docs.juliaplots.org/latest/) plotting tools. They are quite extensive, look nice, and have a variety of [themes](https://docs.juliaplots.org/latest/generated/plotthemes/) and [backends](https://docs.juliaplots.org/latest/backends/) available to create plots similar to other tools (e.g., ggplot).

In [ ]:
# using Pkg; Pkg.add("Plots")
using Plots

plot(df.radius, df.dens, xaxis=:log, yaxis=:log, xlabel="Radius [cm]", ylabel="Density [g/cc]")

In [ ]:
## Can make lots of cool things!
# Example from https://docs.juliaplots.org/latest/

default(legend = false)
x = y = range(-5, 5, length = 40)
zs = zeros(0, 40)
n = 100

@gif for i in range(0, stop = 2π, length = n)
    f(x, y) = sin(x + 10sin(i)) + cos(y)

    # create a plot with 3 subplots and a custom layout
    l = @layout [a{0.7w} b; c{0.2h}]
    p = plot(x, y, f, st = [:surface, :contourf], layout = l)

    # induce a slight oscillating camera angle sweep, in degrees (azimuth, altitude)
    plot!(p[1], camera = (10 * (1 + cos(i)), 40))

    # add a tracking line
    fixed_x = zeros(40)
    z = map(f, fixed_x, y)
    plot!(p[1], fixed_x, y, z, line = (:black, 5, 0.2))
    vline!(p[2], [0], line = (:black, 5))

    # add to and show the tracked values over time
    global zs = vcat(zs, z')
    plot!(p[3], zs, alpha = 0.2, palette = cgrad(:blues).colors)
end

These tools are nice but the controls leave a little bit to be desired for producing publication quality figures. Lucky for us, [PyPlot.jl](https://github.com/JuliaPy/PyPlot.jl) is a wrapper around matplotlib to give us all of our familiar tools.

In [ ]:
# using Pkg; Pkg.add("PyPlot")
using PyPlot # Note that we shouldn't have both Plots and PyPlots loaded. 

In [ ]:
fig, ax = subplots( 1, 2)

ax[1].loglog(df.radius, df.dens, label="label")
ax[2].semilogx(df.radius, df.H1)

ax[1].set(ylabel=L"Density [g cm$^{-3}$]", xlabel = "Radius [cm]")
ax[2].set(ylabel="H Mass Frac", xlabel = "Radius [cm]")
ax[1].legend(frameon=true)

Some notes: There is no `plt.` here. Indexing ax objects starts with 1. Setting things like `marker="o"` _must_ use double quotes -- they're strings, not chars. In Python to get LaTeX string formatting we use `r""`. Here, we use `L""`.

This is my preferred way to creating plots in Julia for now. Julia's default plotting has added a lot of new features since I started and is developing nicely, matplotlib is just much older and more developed for the time being.

# Using Python Libraries / Functions

The great thing baout Python is its huge set of available libraries.
One that we often use is [yt](https://yt-project.org/). Lucky for us, Julia provides a way to (nearly) seamlessly incorporate such tools into our workflow through [PyCall.jl](https://juliapackages.com/p/pycall). We'll use this to examine some `enzo` data with `yt`. These data come from the [yt quickstart guide](https://yt-project.org/doc/quickstart/index.html).

There are two options for using PyCall -- we can use our own Python/Conda installation, or let PyCall manage its own private Conda environment. We'll do the latter here, but I show how to point it to a Python binary on your system.

**Note**: I highly recommend pointing Julia to your Python installation. I've had strange bugs appear with `PyCall` that I can't quite diagnose when using the built in manager.
```julia
using Pkg
ENV["PYTHON"] = "/Users/barker/local/miniconda3/bin/python" # output of "which python"
Pkg.build("PyCall")
```

In [ ]:
# using Pkg; Pkg.add("PyCall")
# using Pkg; Pkg.add("Conda")
# using Conda
# Conda.add("yt")

using PyCall

# ===

In [ ]:
yt = pyimport("yt")
# np_sum = pyimport("numpy")["sum"] # to load just sum from numpy

In [ ]:
# We need to get the data -- too big to put in the repo
# run(`command`) will execute shell commands
run(`curl -sSO https://yt-project.org/data/enzo_tiny_cosmology.tar.gz`)

In [ ]:
run(`tar xzf enzo_tiny_cosmology.tar.gz`)

In [ ]:
# Now we can do the usual things with yt

ds = yt.load("enzo_tiny_cosmology/DD0046/DD0046")
println("Redshift =", ds.current_redshift)

In [ ]:
p = yt.ProjectionPlot(ds, "y", "density")

# Put It All Together

Let's create Python and Julia functions to solve a linear system, benchmark them for various system sizes, and plot the timing results.

In [ ]:
# using Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools
using PyCall

In [ ]:
using LinearAlgebra
using StaticArrays

"""
Create a dense matrix and solve Ax=b, where b is an array of ones.
"""
function LinearSolve( size::Int64 )
    
    matrix :: Matrix{Float64} = ones( size, size )
    matrix += I # Makes it nonsingular
    #Note that Julia knows how to interpret I as the Identity, regardless of size.
    
#     x :: Vector{Float64} = ones( size )
#     b :: Vector{Float64} = matrix * ones( size ) # setup RHS so solution is ones
    
    return (matrix)\ones( size )
end

# LinearSolve.py contains a function doing the same thing using np.linalg.solve()
LinearSolve_Python = pyimport("LinearSolve")

In [ ]:
LinearSolve(3)

In [ ]:
LinearSolve_Python.LinearSolve(3)

In [ ]:
# This takes awhile .. 

time_julia  = []
time_python = []

sizes = [ 4, 16, 64, 128, 256, 512, 1024, 4096, 10000 ]
for i in 1:length(sizes)
    tmp = @benchmark LinearSolve(sizes[$i])
    push!(time_julia, mean(tmp.times) )
    
    tmp = @benchmark LinearSolve_Python.LinearSolve(sizes[$i])
    push!(time_python, mean(tmp.times) )
end

In [ ]:
using PyPlot

fig, ax = subplots()

ax.loglog(sizes, time_julia./1e6, marker="x", ls=" ", label="Julia")
ax.loglog(sizes, time_python./1e6, marker="x", ls=" ", label="Python")

ax.legend(frameon=true)
ax.set(xlabel="N", ylabel="Mean Time [ms]")

# Application Benchmarking

So far we've only benchmarked individual operations -- the linear solver above, summations in the slides in this directory, etc.
Most applications are more complex than just NumPy operations. 
Here we test a finite difference solver for the 1D advection equation
$$
\left( \frac{\partial }{\partial t} + u \frac{\partial }{\partial x}\right) f = 0. 
$$

The Python implementation, found in `advection.py` uses numpy operations for setting up the initial conditions and the evolution steps.
The Julia implementation, foudn in `advection.jl` uses for loops with bounds checking deactivated in place of the equivalent numpy functions.

The implementations used are first order finite difference upwinding schemes.

In [ ]:
py_advection = pyimport("advection")
include("advection.jl")

In [ ]:
@benchmark Solve($128)

In [ ]:
@benchmark py_advection.Solve($100)

In [ ]:
time_julia  = []
time_python = []

sizes = [ 128, 256, 512, 1024, 2048, 4096, 8192 ]
for i in 1:length(sizes)
    tmp = @benchmark Solve(sizes[$i])
    push!(time_julia, mean(tmp.times) )
    
    tmp = @benchmark py_advection.Solve(sizes[$i])
    push!(time_python, mean(tmp.times) )
end

In [ ]:
using PyPlot

fig, ax = subplots()

ax.loglog(sizes, time_julia./1e6, marker="x", ls=" ", label="Julia")
ax.loglog(sizes, time_python./1e6, marker="x", ls=" ", label="Python")
ax.set(xlim=[1e2,1e4])

ax.legend(frameon=true)
ax.set(xlabel="N", ylabel="Mean Time [ms]")

In [ ]:
# try with numba

# externally done and copied. should've written to file and read in but .. *shrugs*
time_numba = [ 5.438017845153808e-05,0.00014248418807983397,0.0004354686737060547,0.0014885380268096925,0.005724998235702515,0.026125758171081544,0.09832416653633118 ]

using PyPlot

fig, ax = subplots()

ax.loglog(sizes, time_julia./1e6, marker="x", ls=" ", label="Julia")
ax.loglog(sizes, time_python./1e6, marker="x", ls=" ", label="Python")
ax.loglog(sizes, time_numba.*1e3, marker="x", ls=" ", label="Numba")
ax.set(xlim=[1e2,1e4])

ax.legend(frameon=true)
ax.set(xlabel="N", ylabel="Mean Time [ms]")